In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:

df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

df_analyse.sort_values('date',ascending=True).tail()

,date,Italy,US,Spain,Germany,"Korea, South"
89,2020-04-20,181228,784326,200210,147065,10674
90,2020-04-21,183957,811865,204178,148291,10683
91,2020-04-22,187327,840351,208389,150648,10694
92,2020-04-23,189973,869170,213024,153129,10708
93,2020-04-24,192994,905358,219764,154999,10718


In [7]:
threshold=100
compare_list=[]
for pos,country in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))

pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])
pd_sync_timelines.head()

,Italy,US,Spain,Germany,"Korea, South",date
0,155.0,118.0,120.0,130.0,104.0,0
1,229.0,149.0,165.0,159.0,204.0,1
2,322.0,217.0,222.0,196.0,433.0,2
3,453.0,262.0,259.0,262.0,602.0,3
4,655.0,402.0,400.0,482.0,833.0,4


In [8]:
fig = make_subplots(rows =2,
                         cols = 2,
                         subplot_titles = ("Infection rate", "Doubling rate"))

trace0 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[US],
                name = US,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"))
trace1 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[US],
                name = US,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"))
trace2 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[US],
                name = US,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 2)

fig.show()

NameError: name 'US' is not defined

In [9]:
trace0 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[each],
                name = each,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"))

trace1 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[each],
                name = each,
                marker = dict(size =12, color = "rgba(155, 70, 0, 0.9)"))

trace2 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[each],
                name = each,
                marker = dict(size =12, color = "rgba(155, 70, 0, 0.9)"))

fig = make_subplots(
    rows =2, cols = 2,
                    
    specs=[[{},{}], [{'colspan':2},None]
          ]
)


fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)

fig.layout["xaxis1"].update(title = "Count")
fig.layout["xaxis2"].update(title = "Count")
fig.layout["xaxis3"].update(title = "Count")

fig.layout["yaxis1"].update(title = "Count")
fig.layout["yaxis2"].update(title = "Count")
fig.layout["yaxis3"].update(title = "Count")
#fig.layout["yaxis4"].update(title = "Count")

fig.layout.update(title = "COVID-19 Data", showlegend = False)

fig.show()

NameError: name 'each' is not defined

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df.head()

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND


In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests

raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])

from datetime import datetime

def convertTime(t):
    t = int(t)
    return datetime.fromtimestamp(t)

data_list = df["attributes"].tolist()
df_final = pd.DataFrame(data_list)
df_final.set_index("OBJECTID")

df_final = df_final[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Last_Update"]]
df_final = df_final.dropna(subset=["Last_Update"])
df_final["Last_Update"]= df_final["Last_Update"]/1000
df_final["Last_Update"] = df_final["Last_Update"].apply(convertTime)
df_final["Province_State"].fillna(value="", inplace=True)

message = df_final["Country_Region"] + " " + df_final["Province_State"] + "<br>"
message += "Confirmed: " + df_final["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df_final["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df_final["Recovered"].astype(str) + "<br>"
message += "Last updated: " + df_final["Last_Update"].astype(str)

df_final["text"] = message

df_total = df_final.groupby("Country_Region", as_index=False).agg(
    {
        "Confirmed" : "sum",
        "Deaths" : "sum",
        "Recovered" : "sum"
    }
)

total_confirmed = df_final["Confirmed"].sum()
total_recovered = df_final["Recovered"].sum()
total_deaths = df_final["Deaths"].sum()

df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()


fig = make_subplots(
    rows = 4, cols = 6,

    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)

fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = df_final["Long_"],
        lat = df_final["Lat"],
        hovertext = df_final["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = df_final['Confirmed'],
            cmax = df_final['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)

fig.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(df_final["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
    geo = dict(
            projection_type="orthographic",
            showcoastlines=True,
            landcolor="white", 
            showland= True,
            showocean = True,
            lakecolor="LightBlue"
    ),

    annotations=[
        dict(
            text="Source: https://bit.ly/3aEzxjK",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.35,
            y=0)
    ]
)

fig.show()

In [4]:
trace0 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[each],
                name = each,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"),
                xaxis = "x",
                yaxis = "y")

trace1 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[each],
                name = each,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"),
                xaxis = "x1",
                yaxis = "y1")

'''trace2 = go.Scatter(x = pd_sync_timelines.date,
                y = pd_sync_timelines[each],
                name = each,
                marker = dict(size =12, color = "rgba(255, 70, 0, 0.9)"),
                xaxis2 = "x2",
                yaxis2  = "y2")'''

data = [trace0, trace1]
layout = go.Layout(title = "COVID-19 Data",
        xaxis=dict(
            domain=[0, 0.6]),

        xaxis1=dict(
            domain=[0.7, 1]),
                   
        yaxis1 = dict(
            anchor='x1')
) 

fig = go.Figure(data= data, layout = layout)
'''fig = tools.make_subplots(rows =2,
                         cols = 2,
                         specs = [[{"rowspan":2}, {}], [{}, {}]],
                         subplot_titles = ("Infection rate", "Doubling rate", "World map"))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
#fig.append_trace(trace2, 2, 2)

fig.layout["xaxis1"].update(title = "Count")
fig.layout["xaxis2"].update(title = "Count")
#fig.layout["xaxis3"].update(title = "Count")

fig.layout["yaxis1"].update(title = "Count")
fig.layout["yaxis2"].update(title = "Count")
#fig.layout["yaxis3"].update(title = "Count")
#fig.layout["yaxis4"].update(title = "Count")

fig.layout.update(title = "COVID-19 Data", showlegend = False)'''

fig.show()

NameError: name 'pd_sync_timelines' is not defined